In [1]:
import numpy as np
import torch
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split

print(os.getcwd())

/home/howc/PycharmProjects/pythonProject2/src


## Train-test split

In [2]:
db = 'test_db/single_near_mutant_CYP3A4_db.csv'
df = pd.read_csv(db)
train_data_df, test_data_df = train_test_split(df, test_size=0.33, random_state=42)

## Get all the Mutein code for training data split

In [3]:
#gonna iterate over all the rows in training data, concatenate mt_code with ligand cid and _complex string

prefix = 'protein_ligand_pdb/xml'
lst = []

for index, row in train_data_df.iterrows():
    mt_code = row[1]
    cid = row[14]
    path_to_xml = f'{prefix}/{mt_code}_{cid}_complex/report.xml'
    lst.append(path_to_xml)

In [4]:
xml_files = set(['protein_ligand_pdb/xml/CYP3A4-S119A_3016_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-T309F_4192_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I301A_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119T_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-F304A_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-R212A_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-T433S_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-F304W_11790_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I369V_4993_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A370V_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4.11_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S312C_4192_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I120L_14403_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-R212A_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A305V_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4.3_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-C98A_4192_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I301A_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-T309F_4192_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-M371I_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119W_3035604_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A305V_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I369V_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4.3_4485_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119A_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I301A_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-T363M_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-L373F_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A370V_4993_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119C_4192_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-F304A_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119F_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I301F_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I301A_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-L373F_4192_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4.3_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A305G_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-L373H_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119A_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I369V_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A370F_55245_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I120L_14403_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-F304A_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119T_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4.3_848780_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4.3_2730_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119L_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-R212A_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-C98F_4192_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I369F_3035604_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A305F_3035604_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A370V_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119L_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A305G_4993_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I301A_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I301A_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-T103A_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-F304A_4192_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119A_55245_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A305V_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119T_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-F304W_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119L_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-T363M_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I120W_4192_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A305V_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A305V_4993_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-V376T_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A305V_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-L373V_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A305G_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A305V_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-M371I_14403_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4.3_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I120F_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-F304W_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-T309V_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-C98W_4192_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119A_55245_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119A_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4.11_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4.11_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-T309C_4192_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119A_55245_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-L373V_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-T309A_3035604_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4.3_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-F304A_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-F304A_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119L_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-L373A_4993_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I120W_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A305V_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A370F_3035604_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A370F_55245_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119F_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119V_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119T_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119A_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I301A_3016_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S312C_4192_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I120L_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-F304A_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119T_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I301F_3016_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-T309C_4192_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119F_3016_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A370F_55245_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-T103A_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4.12_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4.12_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-F304W_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-F304W_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-V376T_14403_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4.12_848780_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119F_3016_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A305G_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-N104D_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4.3_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119F_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I369V_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-N104D_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I369V_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-T309A_848780_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I369V_14403_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-M371I_14403_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A370C_4192_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A305G_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A370V_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-L373F_3035604_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119A_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4.11_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-F304W_4192_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A370C_4192_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-L373F_4192_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I301W_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-L373F_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I369V_4993_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I301F_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A305S_3016_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-T309A_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-T309V_3035604_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119A_3035604_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-A305G_4993_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4.3_2730_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-P107S_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-V376T_14403_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-S119T_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4.11_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-T309A_6013_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I120W_5994_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-L373H_566440_complex/report.xml',
 'protein_ligand_pdb/xml/CYP3A4-I301W_5994_complex/report.xml'])
len(xml_files)

93

In [5]:
len(os.listdir('sample_data'))

93

# Filtering for bad data, where bad is rows of adjacency matrix != rows of feature matrix

In [6]:
data_folder = 'sample_data'
data_set = os.listdir(data_folder)
bad_data = []

for folder in data_set:
    adj_mat_path = f'{data_folder}/{folder}/{folder}_adjmat.npy'
    vertex_feature_mat_path = f'{data_folder}/{folder}/{folder}_featuremat.npy'
    adj_mat = torch.from_numpy(np.load(adj_mat_path)).float()
        
    feature_mat = torch.from_numpy(np.load(vertex_feature_mat_path)).float()

    if adj_mat.shape[0] != feature_mat.shape[0]:
        bad_data.append(folder)

bad_data

['CYP3A4.3_4485_complex']

# Removing bad data

In [7]:
good_data = []

for folder in data_set:
    if folder not in bad_data:
        good_data.append(folder)

good_data

['CYP3A4-F304A_4192_complex',
 'CYP3A4-L373V_566440_complex',
 'CYP3A4-P107S_6013_complex',
 'CYP3A4-I369F_3035604_complex',
 'CYP3A4.11_6013_complex',
 'CYP3A4-T309A_848780_complex',
 'CYP3A4-S119A_3035604_complex',
 'CYP3A4-I369V_14403_complex',
 'CYP3A4.12_5994_complex',
 'CYP3A4-T309C_4192_complex',
 'CYP3A4-T309V_6013_complex',
 'CYP3A4-C98A_4192_complex',
 'CYP3A4-S119W_3035604_complex',
 'CYP3A4-I120W_5994_complex',
 'CYP3A4.3_848780_complex',
 'CYP3A4-T309A_5994_complex',
 'CYP3A4-I301A_566440_complex',
 'CYP3A4-C98W_4192_complex',
 'CYP3A4-S119C_4192_complex',
 'CYP3A4-V376T_6013_complex',
 'CYP3A4-I369V_6013_complex',
 'CYP3A4-A305V_5994_complex',
 'CYP3A4-L373F_4192_complex',
 'CYP3A4-L373A_4993_complex',
 'CYP3A4.3_5994_complex',
 'CYP3A4-I301A_5994_complex',
 'CYP3A4.12_848780_complex',
 'CYP3A4-S312C_4192_complex',
 'CYP3A4-L373F_5994_complex',
 'CYP3A4-T309A_3035604_complex',
 'CYP3A4-F304A_6013_complex',
 'CYP3A4.12_6013_complex',
 'CYP3A4-F304W_11790_complex',
 'CYP3A4

# Network definitions

In [8]:
import torch
import torch.nn.functional as F
import torch.optim as optim

class GraphNN_Attention(torch.nn.Module):
    
    def __init__(self, embedding_dim=2, non_linearity=torch.nn.Sigmoid()):
        
        super(GraphNN_Attention, self).__init__()
        self.embedding_dim = embedding_dim #meaning embedding vector by default is in R^2
        #self.gpu = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        self.W_1 = torch.randn(11, self.embedding_dim, requires_grad=True)
        self.Y_1 = torch.randn(11, self.embedding_dim, requires_grad=True)
        #self.W_cost = torch.randn(self.embedding_dim, 1, requires_grad=True)
        
        self.non_linearity = non_linearity
        
        #self.alpha = torch.nn.Parameter(torch.abs(torch.randn(1, requires_grad=True)))
        self.W_attent = torch.randn(11, 1, requires_grad=True)
        self.complex_embedding_map = {}  #take average of rows of embedding_matrix
        self.atom_embedding_map = {}
        
    def attention_coeff(self, euclid_dist_v_i, i_neighbour_vec):
        #ori_adj_mat contains original euclidean distance info
        i_neighbour_vec = i_neighbour_vec.view(1, i_neighbour_vec.shape[0])
        return (1/euclid_dist_v_i) * torch.mm(i_neighbour_vec, self.W_attent)
    
    def attention_mechanism(self, ori_adj_mat, feature_mat):
        
        attention_coeff_mat = torch.zeros(ori_adj_mat.shape) 
        processed_set = set()
        softmax = torch.nn.Softmax(dim=1)
        
        
        for v in range(ori_adj_mat.shape[0]):

            for i in range(ori_adj_mat.shape[1]):
                
                if v == i:
                    processed_set.add((v, i))
                    continue
                    
                attention_coeff_v_i = self.attention_coeff(ori_adj_mat[v,i], feature_mat[i,:])
                
                attention_coeff_mat[v, i] = attention_coeff_v_i
                processed_set.add((v, i))
        
        attention_mat = softmax(attention_coeff_mat)
        
        return attention_mat

    def forward_gnn(self, adj_mat, feature_mat, complex_code):
        """
            adj_mat (torch tensor): adjacency matrix
            feature_mat (torch tensor): feature_matrix for all atoms
            complex_code (str): cyp_ligand code
            return a tensor of predicted probabilities
            
        """
        #graph nn embedding procedure
        num_neighbours = adj_mat.shape[0] - 1 #since adj mat is square and symmetric
        #whitened_adj_mat = torch.where(adj_mat <= self.alpha, 0, 1).float()
        attent_mat = self.attention_mechanism(adj_mat, feature_mat)
        
        avr_neighbour_feature_linear_comb = (1/num_neighbours) * (torch.mm(attent_mat, feature_mat))
        h_v_matrix = self.non_linearity(torch.mm(avr_neighbour_feature_linear_comb, self.W_1) + torch.mm(feature_mat,self.Y_1))        
        #sigmoid = torch.nn.Sigmoid()
        #prob_tensor = sigmoid(torch.matmul(h_v_matrix, self.W_cost))
        
        ## persistence
        #each row of h_v_matrix == atom embedding, entire matrix == for all atoms
        self.atom_embedding_map[complex_code] = h_v_matrix 
        self.complex_embedding_map[complex_code] = torch.mean(h_v_matrix, dim=0)
        complex_embedding = torch.mean(h_v_matrix, dim=0)
        #return prob_tensor
        return complex_embedding.view(1, complex_embedding.shape[0])

In [9]:
import random

class CNN(torch.nn.Module):
    
    def __init__ (self, output_dim=11):
        
        super(CNN, self).__init__()
        
        self.output_dim = output_dim
        standard_input_shape = (50, 50)
        
        self.adaptive_layer = torch.nn.AdaptiveAvgPool2d(standard_input_shape)
        
        self.first_conv2d = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(5,5))
        self.second_conv2d = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(5,5))
        
        self.elu = torch.nn.ELU(inplace=False)
        self.avg_pooling = torch.nn.AvgPool2d(5, stride=1)
        
        self.contact_map_embeddings_map = {}
        self.fnn = torch.nn.Linear(in_features=361, out_features=self.output_dim)
        
    def convolution_cnn(self, tensor_2d_contact_map):
        
        contact_map_tensor = tensor_2d_contact_map.reshape(1, 1, tensor_2d_contact_map.shape[0], tensor_2d_contact_map.shape[1])
        
        ### standardisation routine
        standardised_contact_map_tensor = self.adaptive_layer(contact_map_tensor).float()
        
        ### convolution routine
        first_conv_out = self.first_conv2d(standardised_contact_map_tensor)
        first_elu_out = self.elu(first_conv_out)
        
        ### 2nd convolution routine
        second_conv_out = self.second_conv2d(first_elu_out)
        second_elu_out = self.elu(second_conv_out)
        
        ### average pooling
        avg_pooled_img = self.avg_pooling(second_elu_out)
        
        ### downsampling
        down_sampled_img = torch.nn.functional.interpolate(avg_pooled_img, scale_factor=0.5, recompute_scale_factor=True)
        
        ### feed-forward network (do a linear transformation)
        vectorised_img = down_sampled_img.view(1, -1)
        num_img_features = vectorised_img.shape[-1]
        
        img_vector = self.fnn(vectorised_img)
        
        return img_vector
        
    def forward_cnn(self, tensor_2d_contact_map, mt_code):
        
        contact_map_embeddings = self.convolution_cnn(tensor_2d_contact_map)
        self.contact_map_embeddings_map[mt_code] = contact_map_embeddings
        
        return contact_map_embeddings 

In [10]:
import random

class RegressionNN(torch.nn.Module):
    def __init__(self, dim=(1,22)):
        """
            dim (int) should be the shape of contact_complex_embedding
        """
        super(RegressionNN, self).__init__()
        self.first_out = random.randint(1, 10)
        self.dim = dim
        self.first_layer = torch.nn.Linear(in_features=self.dim[-1], out_features=self.first_out)
        self.first_non_linear = torch.nn.Tanh()
        self.second_layer = torch.nn.Linear(in_features=self.first_out, out_features=1)
        
    def forward_reg(self, contact_complex_embedding):
        first_layer_out = self.first_layer(contact_complex_embedding)
        first_non_linear_out = self.first_non_linear(first_layer_out)
        result = self.second_layer(first_non_linear_out)
        return result

In [11]:
class CombinedNN(torch.nn.Module):
    
    def __init__(self, gnn, cnn, regressor):
        super(CombinedNN, self).__init__()
        self.gnn = gnn
        self.cnn = cnn
        self.regressor = regressor
    
    def forward(self, adj_mat, feature_mat, complex_code, tensor_2d_contact_map, mt_code):
        complex_atoms_embedding = self.gnn.forward_gnn(adj_mat, feature_mat, complex_code)
        contact_maps_embedding = self.cnn.forward_cnn(tensor_2d_contact_map, mt_code)
                
        contact_complex_embedding = torch.cat((complex_atoms_embedding, contact_maps_embedding), dim=1)
        
        regression_pred = self.regressor.forward_reg(contact_complex_embedding)
        
        return regression_pred

In [12]:
CombinedNNfrom torch.utils.data import Dataset, DataLoader
import os

class ComplexDataset(Dataset):
    
    def __init__(self, data_dir, good_data_lst, db):
        self.data_dir = data_dir
        self.good_data_lst = good_data_lst # a numpy array of good data, after slicing with ids from KFold
        self.db = db # db data_frame
        
    def __getitem__(self, index):
        curr_good_data = self.good_data_lst[index]
        
        #gnn data paths
        adj_mat_path = f'{self.data_dir}/{curr_good_data}/{curr_good_data}_adjmat.npy'
        vertex_feature_mat_path = f'{self.data_dir}/{curr_good_data}/{curr_good_data}_featuremat.npy'
        
        ## gnn data
        feature_mat = torch.from_numpy(np.load(vertex_feature_mat_path)).float()
        adj_mat = torch.from_numpy(np.load(adj_mat_path)).float()
        
        atom_to_adj_row = json.load(open(f'{self.data_dir}/{curr_good_data}/{curr_good_data}_atom_to_row_adj_mat.json'))
        atom_to_feature_row = json.load(open(f'{self.data_dir}/{curr_good_data}/{curr_good_data}_atom_to_row_vertex_feature_mat.json'))
        
        ## cnn data
        mt_code = curr_good_data.split('_')[0]
        contact_map_path = f'2DContactMaps/CYP3A4_Mutants_Binding/{mt_code}.npy'
        contact_map_np = np.load(contact_map_path)
        
        ## regression data
        ligand_id = int(curr_good_data.split('_')[1])
        filtered_data = self.db[(self.db['mt_code'] == mt_code) & (self.db['cid'] == ligand_id)]
        true_activity = torch.tensor(float(filtered_data.iloc[0, 11])).view(1,1)
        
        return feature_mat, adj_mat, atom_to_adj_row, atom_to_feature_row, mt_code, torch.from_numpy(contact_map_np), true_activity, curr_good_data
    
    def __len__(self):
        return len(self.good_data_lst)

# Slow training code

In [38]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from statistics import mean

## hyper-parameters
data_base = 'test_db/single_near_mutant_CYP3A4_db.csv'
db_df = pd.read_csv(data_base)

epochs = 20000
learning_rate = 0.02

regression_loss = torch.nn.MSELoss()
data_dir = 'sample_data'

## k-fold
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True)

## model
gnn = GraphNN_Attention(embedding_dim=11)
cnn = CNN()
regressor = RegressionNN()
combined_nn = CombinedNN(gnn, cnn, regressor)

gnn_params = [gnn.W_1, gnn.Y_1, gnn.W_attent]
cnn_params = list(cnn.parameters())
regressor_params = list(regressor.parameters())

all_params = gnn_params + cnn_params + regressor_params

optimiser = torch.optim.Adam(all_params, lr=learning_rate)


training_losses = [] #mean_epoch_i(train_mse_fold1, train_mse_fold2,...,train_mse_foldn)
cv_lossess = []      #mean_epoch_i(cv_mse_fold1, cv_mse_fold2,...,cv_mse_foldn)

training_stats_log = 'training_statistics/loss.txt'

for epoch in range(epochs):
    
    training_loss_all_folds = []
    cv_loss_all_folds = []
    
    for fold, (train_ids, cv_ids) in enumerate(kfold.split(good_data)):
        
        trng_data_amt = len(train_ids)
        cv_data_amt = len(cv_ids)
        total_trng_loss_curr_fold = 0
        
        for train_id in train_ids:
            
            folder = good_data[train_id]
            #GNN related data
            adj_mat_path = f'{data_dir}/{folder}/{folder}_adjmat.npy'
            vertex_feature_mat_path = f'{data_dir}/{folder}/{folder}_featuremat.npy'
            adj_mat = torch.from_numpy(np.load(adj_mat_path)).float()
            print(f'adj_mat: {adj_mat.shape}')
            
            feature_mat = torch.from_numpy(np.load(vertex_feature_mat_path)).float()
            print(feature_mat.shape)
            atom_to_adj_row = json.load(open(f'{data_dir}/{folder}/{folder}_atom_to_row_adj_mat.json'))
            atom_to_feature_row = json.load(open(f'{data_dir}/{folder}/{folder}_atom_to_row_vertex_feature_mat.json'))
        
            # CNN related data
            mt_code = folder.split('_')[0]
            contact_map_path = f'2DContactMaps/CYP3A4_Mutants_Binding/{mt_code}.npy'
            contact_map_np = np.load(contact_map_path)
        
            with torch.autograd.set_detect_anomaly(True):
                # 1.gnn + cnn + regression
                pred_activity = combined_nn.forward(adj_mat, feature_mat, folder, torch.from_numpy(contact_map_np), mt_code)
                    
                # 2.regression: getting ground truth label based on mt code
                ligand_id = int(folder.split('_')[1])
                filtered_data = db_df[(db_df['mt_code'] == mt_code) & (db_df['cid'] == ligand_id)]
                true_activity = float(filtered_data.iloc[0, 11])
                true_activity = torch.tensor(true_activity).view(1,1)
            
                train_loss = regression_loss(pred_activity, true_activity)
                train_loss.backward()
                optimiser.step()
                optimiser.zero_grad()            
                total_trng_loss_curr_fold += train_loss.item()
        
        training_loss_all_folds.append(total_trng_loss_curr_fold/trng_data_amt) #persis the mean of all trng losses

        total_cv_loss_curr_fold = 0
        
        for cv_id in cv_ids:
            cv_folder = good_data[cv_id]
            
            #gnn data
            adj_mat_path = f'{data_dir}/{cv_folder}/{cv_folder}_adjmat.npy'
            vertex_feature_mat_path = f'{data_dir}/{cv_folder}/{cv_folder}_featuremat.npy'
            adj_mat = torch.from_numpy(np.load(adj_mat_path)).float()
            
            feature_mat = torch.from_numpy(np.load(vertex_feature_mat_path)).float()
            atom_to_adj_row = json.load(open(f'{data_dir}/{cv_folder}/{cv_folder}_atom_to_row_adj_mat.json'))
            atom_to_feature_row = json.load(open(f'{data_dir}/{cv_folder}/{cv_folder}_atom_to_row_vertex_feature_mat.json'))
            
            #cnn data
            mt_code = cv_folder.split('_')[0]
            contact_map_path = f'2DContactMaps/CYP3A4_Mutants_Binding/{mt_code}.npy'
            contact_map_np = np.load(contact_map_path)
            
            with torch.no_grad():
                pred_cv = combined_nn.forward(adj_mat, feature_mat, folder, torch.from_numpy(contact_map_np), mt_code)
 
       
        cv_loss_all_folds.append(total_cv_loss_curr_fold/cv_data_amt)
    
    training_losses.append(mean(training_loss_all_folds))
    cv_lossess.append(mean(cv_loss_all_folds))
    
    with open(training_stats_log, 'a+') as f:
        f.write(f'epoch_{epoch+1},trng_MSE_{mean(training_loss_all_folds)},cv_MSE_{mean(cv_loss_all_folds)} \n')
                    
    print(f'epoch {epoch+1} trng MSE: {mean(training_loss_all_folds)} cv MSE: {mean(cv_loss_all_folds)}')

adj_mat: torch.Size([15, 15])
torch.Size([15, 11])
torch.Size([15, 15])
torch.Size([15, 11])
(0, 1)
tensor([3., 0., 0., 1., 0., 1., 0., 0., 2., 0., 0.])
tensor(3.5507)
(0, 2)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 14.,  0.,  0.])
tensor(3.3801)
(0, 3)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 14.,  0.,  0.])
tensor(3.5760)
(0, 4)
tensor([2., 0., 1., 0., 0., 2., 0., 0., 0., 0., 1.])
tensor(8.0782)
(0, 5)
tensor([2., 0., 1., 0., 0., 1., 0., 0., 2., 0., 1.])
tensor(8.9671)
(0, 6)
tensor([2., 0., 0., 0., 0., 2., 0., 0., 0., 0., 1.])
tensor(5.9359)
(0, 7)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(8.8607)
(0, 8)
tensor([2., 0., 0., 0., 0., 1., 0., 0., 2., 0., 1.])
tensor(10.1639)
(0, 9)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(10.3005)
(0, 10)
tensor([2., 0., 0., 0., 0., 1., 0., 0., 2., 0., 1.])
tensor(8.7133)
(0, 11)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(8.1388)
(0, 12)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

tensor([2., 0., 0., 0., 0., 1., 0., 0., 6., 0., 1.])
tensor(8.4008)
(12, 0)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(9.3594)
(12, 1)
tensor([3., 0., 0., 1., 0., 1., 0., 0., 2., 0., 0.])
tensor(10.7414)
(12, 2)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 14.,  0.,  0.])
tensor(11.2173)
(12, 3)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 14.,  0.,  0.])
tensor(9.8364)
(12, 4)
tensor([2., 0., 1., 0., 0., 2., 0., 0., 0., 0., 1.])
tensor(2.2564)
(12, 5)
tensor([2., 0., 1., 0., 0., 1., 0., 0., 2., 0., 1.])
tensor(5.4282)
(12, 6)
tensor([2., 0., 0., 0., 0., 2., 0., 0., 0., 0., 1.])
tensor(3.7345)
(12, 7)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(5.0643)
(12, 8)
tensor([2., 0., 0., 0., 0., 1., 0., 0., 2., 0., 1.])
tensor(5.4550)
(12, 9)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(4.2302)
(12, 10)
tensor([2., 0., 0., 0., 0., 1., 0., 0., 2., 0., 1.])
tensor(3.9054)
(12, 11)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(3.

tensor([3., 0., 0., 1., 0., 1., 0., 0., 2., 0., 0.])
tensor(8.2423)
(2, 3)
tensor([3., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.])
tensor(3.2855)
(2, 4)
tensor([1., 0., 1., 0., 0., 2., 0., 0., 0., 0., 1.])
tensor(7.4491)
(2, 5)
tensor([2., 0., 1., 0., 0., 1., 0., 0., 2., 0., 0.])
tensor(10.5807)
(2, 6)
tensor([1., 0., 0., 0., 0., 2., 0., 0., 0., 0., 1.])
tensor(4.8131)
(2, 7)
tensor([2., 0., 0., 0., 0., 1., 0., 0., 2., 0., 1.])
tensor(4.2034)
(2, 8)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(4.8816)
(2, 9)
tensor([1., 0., 0., 0., 0., 1., 0., 0., 6., 0., 1.])
tensor(8.6135)
(2, 10)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(7.8369)
(3, 0)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(7.5354)
(3, 1)
tensor([3., 0., 0., 1., 0., 1., 0., 0., 2., 0., 0.])
tensor(10.7001)
(3, 2)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(3.2855)
(3, 4)
tensor([1., 0., 1., 0., 0., 2., 0., 0., 0., 0., 1.])
tensor(9.2083)
(3, 5)
tensor([2., 0., 1., 0.

(3, 10)
tensor([1., 0., 0., 0., 0., 2., 0., 0., 0., 0., 1.])
tensor(10.7715)
(3, 11)
tensor([ 1.,  0.,  0.,  0.,  0.,  1.,  0.,  0., 20.,  0.,  1.])
tensor(7.0642)
(3, 12)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(7.6507)
(3, 13)
tensor([ 2.,  0.,  0.,  0.,  0.,  1.,  0.,  0., 11.,  0.,  1.])
tensor(7.5516)
(4, 0)
tensor([1., 0., 1., 0., 0., 2., 0., 0., 0., 0., 0.])
tensor(4.7793)
(4, 1)
tensor([2., 0., 1., 0., 0., 1., 0., 0., 2., 0., 0.])
tensor(5.5589)
(4, 2)
tensor([1., 0., 1., 0., 0., 2., 0., 0., 0., 0., 0.])
tensor(2.3890)
(4, 3)
tensor([ 1.,  0.,  1.,  0.,  0.,  1.,  0.,  0., 16.,  0.,  1.])
tensor(4.4888)
(4, 5)
tensor([2., 0., 0., 0., 0., 1., 0., 0., 2., 0., 1.])
tensor(4.2606)
(4, 6)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(3.9823)
(4, 7)
tensor([1., 0., 0., 0., 0., 1., 0., 0., 6., 0., 1.])
tensor(4.7807)
(4, 8)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(3.6182)
(4, 9)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
te

(2, 3)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 14.,  0.,  0.])
tensor(3.8735)
(2, 4)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(2.3535)
(2, 5)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 14.,  0.,  0.])
tensor(4.3270)
(2, 6)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 13.,  0.,  0.])
tensor(5.5469)
(2, 7)
tensor([3., 0., 0., 1., 0., 1., 0., 0., 2., 0., 0.])
tensor(5.8354)
(2, 8)
tensor([1., 0., 0., 0., 0., 2., 0., 0., 0., 0., 1.])
tensor(9.7001)
(2, 9)
tensor([ 1.,  0.,  0.,  0.,  0.,  1.,  0.,  0., 16.,  0.,  1.])
tensor(12.4791)
(2, 10)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(9.8810)
(3, 0)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(6.1740)
(3, 1)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 14.,  0.,  0.])
tensor(7.3260)
(3, 2)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(3.8735)
(3, 4)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(3.7747)
(3, 5)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.

...

(5, 3)
tensor([1., 0., 0., 0., 0., 1., 0., 0., 2., 0., 1.])
tensor(2.8065)
(5, 4)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(2.8798)
(5, 6)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(9.2220)
(5, 7)
tensor([1., 0., 0., 0., 0., 2., 0., 0., 0., 0., 1.])
tensor(6.0300)
(5, 8)
tensor([2., 0., 0., 0., 0., 1., 0., 0., 2., 0., 1.])
tensor(11.3275)
(5, 9)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(8.8303)
(5, 10)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(10.2818)
(6, 0)
tensor([1., 0., 1., 0., 0., 2., 0., 0., 0., 0., 1.])
tensor(4.0783)
(6, 1)
tensor([ 1.,  0.,  1.,  0.,  0.,  1.,  0.,  0., 16.,  0.,  1.])
tensor(3.2015)
(6, 2)
tensor([1., 0., 0., 0., 0., 2., 0., 0., 0., 0., 1.])
tensor(5.5107)
(6, 3)
tensor([1., 0., 0., 0., 0., 1., 0., 0., 2., 0., 1.])
tensor(7.8462)
(6, 4)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(8.1846)
(6, 5)
tensor([2., 0., 0., 0., 0., 1., 0., 0., 2., 0., 1.])
tensor(9.2220)
(6, 7)
tens

tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(1.5250)
(5, 11)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 14.,  0.,  0.])
tensor(4.1163)
(5, 12)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(7.7293)
(5, 13)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 10.,  0.,  0.])
tensor(11.2796)
(5, 14)
tensor([1., 0., 0., 0., 0., 2., 0., 0., 0., 0., 1.])
tensor(4.7498)
(5, 15)
tensor([ 1.,  0.,  0.,  0.,  0.,  1.,  0.,  0., 16.,  0.,  1.])
tensor(7.7515)
(5, 16)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(5.8719)
(5, 17)
tensor([1., 0., 0., 0., 0., 2., 0., 0., 0., 0., 1.])
tensor(8.8097)
(5, 18)
tensor([ 2.,  0.,  0.,  0.,  0.,  1.,  0.,  0., 11.,  0.,  1.])
tensor(11.7426)
(5, 19)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor(11.3639)
(6, 0)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(6.8764)
(6, 1)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 14.,  0.,  0.])
tensor(5.9141)
(6, 2)
tensor([ 3.,  0.,  0.,  1.,  0., 

tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 10.,  0.,  0.])
tensor(4.5853)
(14, 3)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(9.4531)
(14, 4)
tensor([3., 0., 0., 1., 0., 1., 0., 0., 2., 0., 0.])
tensor(10.0355)
(14, 5)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(4.7498)
(14, 6)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 14.,  0.,  0.])
tensor(8.0062)
(14, 7)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(6.9494)
(14, 8)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 14.,  0.,  0.])
tensor(7.4521)
(14, 9)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 14.,  0.,  0.])
tensor(7.1027)
(14, 10)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(4.4527)
(14, 11)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 14.,  0.,  0.])
tensor(6.3136)
(14, 12)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(9.2799)
(14, 13)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 10.,  0.,  0.])
tensor(12.2252)
(14, 15)
tensor([ 1.,  0

(4, 9)
tensor([2., 0., 1., 0., 0., 1., 0., 0., 2., 0., 0.])
tensor(10.1593)
(5, 0)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(4.5538)
(5, 1)
tensor([3., 0., 0., 1., 0., 1., 0., 0., 2., 0., 0.])
tensor(7.8941)
(5, 2)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(4.6421)
(5, 3)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 10.,  0.,  0.])
tensor(2.5272)
(5, 4)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(3.6834)
(5, 6)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(9.7652)
(5, 7)
tensor([ 3.,  0.,  0.,  1.,  0.,  1.,  0.,  0., 14.,  0.,  0.])
tensor(11.1247)
(5, 8)
tensor([1., 0., 1., 0., 0., 2., 0., 0., 0., 0., 0.])
tensor(9.9817)
(5, 9)
tensor([2., 0., 1., 0., 0., 1., 0., 0., 2., 0., 0.])
tensor(11.4723)
(6, 0)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(5.6993)
(6, 1)
tensor([3., 0., 0., 1., 0., 1., 0., 0., 2., 0., 0.])
tensor(5.5089)
(6, 2)
tensor([3., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.])
tensor(5.2954)


KeyboardInterrupt: 

In [14]:
gnn = GraphNN_Attention(embedding_dim=11)
gnn.W_1.is_cuda

False

In [15]:
torch.cuda.is_available()

True

# Attempt to improve training efficiency

In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from statistics import mean

## hyper-parameters
data_base = 'test_db/single_near_mutant_CYP3A4_db.csv'
db_df = pd.read_csv(data_base)

epochs = 20000
learning_rate = 0.02

regression_loss = torch.nn.MSELoss()
good_data = np.array(good_data)

## k-fold
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True)

## model
gnn = GraphNN_Attention(embedding_dim=11)
cnn = CNN()
regressor = RegressionNN()
combined_nn = CombinedNN(gnn, cnn, regressor)

gnn_params = [gnn.W_1, gnn.Y_1, gnn.W_attent]
cnn_params = list(cnn.parameters())
regressor_params = list(regressor.parameters())

all_params = gnn_params + cnn_params + regressor_params

optimiser = torch.optim.Adam(all_params, lr=learning_rate)


training_losses = [] #mean_epoch_i(train_mse_fold1, train_mse_fold2,...,train_mse_foldn)
cv_lossess = []      #mean_epoch_i(cv_mse_fold1, cv_mse_fold2,...,cv_mse_foldn)

training_stats_log = 'training_statistics/loss.txt'

for epoch in range(epochs):
    
    train_mse_all_folds = []
    cv_mse_all_folds = []
    
    for fold, (train_ids, cv_ids) in enumerate(kfold.split(good_data)):
        
        trng_data_amt = len(train_ids)
        cv_data_amt = len(cv_ids)
        
        trng_data = ComplexDataset(data_dir='sample_data', good_data_lst=good_data[train_ids], db=db_df)
        val_data = ComplexDataset(data_dir='sample_data', good_data_lst=good_data[cv_ids], db=db_df)
        
        train_dataloader = DataLoader(trng_data, batch_size=1, shuffle=True, num_workers=4)
        val_dataloader = DataLoader(val_data, batch_size=1, shuffle=True, num_workers=4)
        
        curr_fold_train_mse = 0
        #forward
        for ith_train_id, (feature_mat, adj_mat, atom_to_adj_row, atom_to_feature_row, mt_code, contact_map, true_activity, folder) in enumerate(train_dataloader):
            
            adj_mat = adj_mat.view(adj_mat.shape[1], adj_mat.shape[2])
            feature_mat = feature_mat.view(feature_mat.shape[1], feature_mat.shape[2])
            contact_map = contact_map.view(contact_map.shape[1], contact_map.shape[2])
            pred_activity = combined_nn.forward(adj_mat, feature_mat, folder[0], contact_map, mt_code[0])
            
            train_loss = regression_loss(pred_activity, true_activity)
            train_loss.backward()
            curr_fold_train_mse += train_loss.item()
            optimiser.step()
            optimiser.zero_grad()
        train_mse_all_folds.append(curr_fold_train_mse/trng_data_amt)
        
        curr_fold_cv_mse = 0
        #cross validation
        for ith_cv_id, (feature_mat, adj_mat, atom_to_adj_row, atom_to_feature_row, mt_code, contact_map, true_cv, folder) in enumerate(train_dataloader):
            adj_mat = adj_mat.view(adj_mat.shape[1], adj_mat.shape[2])
            feature_mat = feature_mat.view(feature_mat.shape[1], feature_mat.shape[2])
            contact_map = contact_map.view(contact_map.shape[1], contact_map.shape[2])
            with torch.no_grad():
                pred_cv = combined_nn.forward(adj_mat, feature_mat, folder[0], contact_map, mt_code[0])
                cv_loss = regression_loss(pred_cv, true_cv)
                curr_fold_cv_mse += cv_loss.item()
        cv_mse_all_folds.append(curr_fold_cv_mse/cv_data_amt)
    
    with open(training_stats_log, 'a+') as f:
        f.write(f'epoch_{epoch+1},trng_MSE_{mean(train_mse_all_folds)},cv_MSE_{mean(cv_mse_all_folds)} \n')
        
    print(f'epoch_{epoch+1},trng_MSE_{mean(train_mse_all_folds)},cv_MSE_{mean(cv_mse_all_folds)}')

/home/howc/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1, 1, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


KeyboardInterrupt: 

In [ ]:
with open(training_stats_log, 'r') as f:
    lines = f.readlines()
lines

# Overfitting on single example

In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from statistics import mean

## hyper-parameters
data_base = 'test_db/single_near_mutant_CYP3A4_db.csv'
db_df = pd.read_csv(data_base)

epochs = 1000
learning_rate = 0.02

regression_loss = torch.nn.MSELoss()
data_dir = 'sample_data'

## model
gnn = GraphNN_Attention(embedding_dim=11)
cnn = CNN()
regressor = RegressionNN()
combined_nn = CombinedNN(gnn, cnn, regressor)

gnn_params = [gnn.W_1, gnn.Y_1, gnn.W_attent]
cnn_params = list(cnn.parameters())
regressor_params = list(regressor.parameters())

all_params = gnn_params + cnn_params + regressor_params

optimiser = torch.optim.Adam(all_params, lr=learning_rate)

for epoch in range(epochs):
    curr_epoch_mse = 0
    
    for folder in os.listdir(data_dir):
        if folder == 'CYP3A4.3_4485_complex':
            continue
        #GNN related data
        adj_mat_path = f'{data_dir}/{folder}/{folder}_adjmat.npy'
        vertex_feature_mat_path = f'{data_dir}/{folder}/{folder}_featuremat.npy'
        adj_mat = torch.from_numpy(np.load(adj_mat_path)).float()
        
        feature_mat = torch.from_numpy(np.load(vertex_feature_mat_path)).float()
        atom_to_adj_row = json.load(open(f'{data_dir}/{folder}/{folder}_atom_to_row_adj_mat.json'))
        atom_to_feature_row = json.load(open(f'{data_dir}/{folder}/{folder}_atom_to_row_vertex_feature_mat.json'))
        
        # CNN related data
        mt_code = folder.split('_')[0]
        contact_map_path = f'2DContactMaps/CYP3A4_Mutants_Binding/{mt_code}.npy'
        contact_map_np = np.load(contact_map_path)
        
        with torch.autograd.set_detect_anomaly(True):
            # 1.gnn + cnn + regression
            pred_activity = combined_nn.forward(adj_mat, feature_mat, folder, torch.from_numpy(contact_map_np), mt_code)
                    
            # 2.regression: getting ground truth label based on mt code
            ligand_id = int(folder.split('_')[1])
            filtered_data = db_df[(db_df['mt_code'] == mt_code) & (db_df['cid'] == ligand_id)]
            true_activity = float(filtered_data.iloc[0, 11])
            true_activity = torch.tensor(true_activity).view(1,1)
            
            train_loss = regression_loss(pred_activity, true_activity)
            train_loss.backward()
            optimiser.step()
            optimiser.zero_grad()            
            curr_epoch_mse += train_loss.item()
        
        count += 1
    print(f'epoch {epoch+1} trng MSE: {curr_epoch_mse/92}')

epoch 1 trng MSE: 2571.381619189856
epoch 2 trng MSE: 2502.3059540162085
epoch 3 trng MSE: 2445.301286041131
epoch 4 trng MSE: 2402.8560235209093


KeyboardInterrupt: 

# Debugging NN

In [26]:
def pca(mat, n_components=2):
    
    from sklearn import preprocessing
    from sklearn.decomposition import PCA
    
    scaled_embeddings_matrix = preprocessing.StandardScaler().fit_transform(mat)
    
    pca = PCA(n_components=n_components)
    embeddings_matrix_pca = pca.fit_transform(scaled_embeddings_matrix)
    
    return embeddings_matrix_pca

def check_bond_info(np_vec):
    bond_info = []
    for i in range(len(np_vec)):
        if i == 1 or i == 2 or i == 3 or i == 4 or i == 6 or i ==7 or i ==9 or i ==10:
            if np_vec[1] == 1:
                bond_info.append('halogen_bonds')
                
            if np_vec[2] == 1:
                bond_info.append('h2_bonds')
            
            if np_vec[3] == 1:
                bond_info.append('hydrophobic_int')
            
            if np_vec[4] == 1:
                bond_info.append('metal complex')
            
            if np_vec[6] == 1:
                bond_info.append('pi cation int')
            
            if np_vec[7] == 1:
                bond_info.append('pi stack')
            
            if np_vec[9] == 1:
                bond_info.append('salt bridge')
            
            if np_vec[10] == 1:
                bond_info.append('h2o bridge')
    bond_info = list(set(bond_info))
    return '/'.join(bond_info)

def prep_pandas_df(embeddings_mat, columns, atom_to_feature_row, feature_mat):
    import pandas as pd
    embeddings_df = pd.DataFrame(embeddings_mat, columns = columns)
    atom_names = []
    bond_info = []
    for index, row in embeddings_df.iterrows():
        
        atom_names.append(atom_to_feature_row[str(index)])
        feature_vec = feature_mat.numpy()[index, :]
        bond_info.append(check_bond_info(feature_vec))

    embeddings_df['atoms'] = atom_names
    embeddings_df['bond_info'] = bond_info
    
    return embeddings_df

def tsne(mat, perplexity=5, n_components=2):
    from sklearn.manifold import TSNE
    return TSNE(n_components=n_components, perplexity=perplexity).fit_transform(mat)

data_dir = 'sample_data'
pca_df_map = {}
tsne_df_map = {}

for complex_id in gnn.atom_embedding_map:
    vertex_feature_mat_path = f'{data_dir}/{complex_id}/{complex_id}_featuremat.npy'
    atom_to_feat_row = json.load(open(f'{data_dir}/{complex_id}/{complex_id}_atom_to_row_vertex_feature_mat.json'))
    feature_mat = torch.from_numpy(np.load(vertex_feature_mat_path)).float()
    
    embeddings_pca_mat = pca(gnn.atom_embedding_map[complex_id].detach().numpy())
    pca_embeddings_df = prep_pandas_df(embeddings_pca_mat, ['x', 'y'], atom_to_feat_row, feature_mat)
    
    embeddings_tsne_mat = tsne(gnn.atom_embedding_map[complex_id].detach().numpy())
    tsne_embeddings_df = prep_pandas_df(embeddings_tsne_mat, ['x', 'y'], atom_to_feat_row, feature_mat)
    
    pca_df_map[complex_id] = pca_embeddings_df
    tsne_df_map[complex_id] = tsne_embeddings_df

In [24]:
import plotly.express as px

fig = px.scatter(df_map['CYP3A4.12_848780_complex'], x="x", y="y", color="bond_info",
                title=f"PCA projected Atoms embedding for {folder}")
fig.show()

In [33]:
import plotly.express as px

tsne_fig = px.scatter(tsne_df_map['CYP3A4-T309V_6013_complex'], x="x", y="y", color="bond_info",
                title=f"TSNE projected Atoms embedding for {folder}")
tsne_fig.show()

In [55]:
tensors = []
id_to_row_map = {}

i = 0
for complex_id in gnn.complex_embedding_map:
    
    tensors.append(gnn.complex_embedding_map[complex_id])
    next_row = gnn.complex_embedding_map[complex_id]
    id_to_row_map[complex_id] = i
    i += 1

all_complex_tensor = torch.stack(tensors, dim=0)
all_complex_tensor
id_to_row_map

{'CYP3A4-T309V_6013_complex': 0,
 'CYP3A4-V376T_6013_complex': 1,
 'CYP3A4.12_848780_complex': 2}

In [56]:
def prep_pandas_complex_df(embeddings_mat, columns, id_to_row_map):
    
    import pandas as pd
    embeddings_df = pd.DataFrame(embeddings_mat, columns = columns)
    complex_id = []
    
    for index, row in embeddings_df.iterrows():
        
        for id in id_to_row_map:
            if id_to_row_map[id] == index:
                complex_id.append(id)
                break
    
    embeddings_df['complex_id'] = complex_id

    return embeddings_df

In [58]:
embeddings_tsne_mat = tsne(all_complex_tensor.detach().numpy())
complex_df = prep_pandas_complex_df(embeddings_tsne_mat, ['x', 'y'], id_to_row_map)

In [59]:
import plotly.express as px

tsne_fig = px.scatter(complex_df, x="x", y="y", color="complex_id",
                title=f"TSNE projected complex embeddings")
tsne_fig.show()